In [5]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import gspread as gs
import json
from datetime import timedelta, date, datetime as dt
import os

print(os.getcwd())

today = dt.now().strftime("%m/%d/%Y %I:%M%p")
t_stamp = dt.now().strftime("%Y-%m-%d %H:%M:%S")

parameters_json = open("../.creds/parameters.json")
json_ = "../.creds/budgetwithme.json"
params = json.load(parameters_json)

url = "https://docs.google.com/spreadsheets/d/{0}/edit?usp=sharing".format(params['sheet_id'])



/Users/enricilagan/Github/email_send/main


In [84]:
gc = gs.service_account(filename=json_)
sh = gc.open_by_url(url)


In [97]:
budget_ws = sh.worksheet('Summary')
budget = pd.DataFrame(budget_ws.get("B3:D7"))

df_budget = budget.iloc[2:,:].reset_index(drop=True)
df_budget.columns = ['Category', 'Budget', 'Remaining']
df_budget['Budget'] = df_budget['Budget'].str.replace("$", "", regex=False).str\
    .replace(",", "", regex=False).astype(float)
df_budget['Remaining'] = df_budget['Remaining'].str.replace("$", "", regex=False).str\
    .replace(",", "", regex=False).astype(float).round(2)
#df_budget['Budget'] = "$" + df_budget['Budget'].astype(str)
#df_budget['Budget'] = "$" + df_budget['Budget'].astype(str)


In [113]:
df_budget

,Category,Budget,Remaining
0,Funds - Marian,100.0,10.69
1,Funds - Enric,80.0,-5.90
2,Funds - Ethan,120.0,11.98


In [134]:
def highlights(s, budget, remaining):
    is_red = pd.Series(data=False, index=s.index)
    if s.loc[remaining]/s.loc[budget] < 0:
        color = 'rgb(222, 37, 37)'
    elif s.loc[remaining]/s.loc[budget] < 0.25:
        color = 'lightcoral'
    elif s.loc[remaining]/s.loc[budget] < 0.5:
        color = 'lightgoldenyellow'
    elif s.loc[remaining]/s.loc[budget] < 0.75:
        color = 'rgb(237, 184, 87)'
    else:
        color = 'rgb(65, 206, 74)'
    #ret = []
    #for v in is_red:
    #    if v[budget]:
    #        ret.append('background-color: rgb(218, 79, 79)') 
    #    else:
    #        ret.append('')
    return [f'background-color: {color}' for v in is_red]

#rgb(227, 37, 37)
df = df_budget.style.apply(highlights, budget="Budget",remaining="Remaining", axis=1)
df

,Category,Budget,Remaining
0,Funds - Marian,100.000000,10.690000
1,Funds - Enric,80.000000,-5.900000
2,Funds - Ethan,120.000000,11.980000


In [129]:
is_red = pd.Series(data=False)
is_red[100] = False



In [130]:
['background-color: rgb(218, 79, 79)' if is_red[100] else '' for v in is_red]

['', '']